In [5]:
import json
import pandas as pd
import geopandas as gpd
from shapely import wkt
from sklearn.preprocessing import MinMaxScaler
import folium
from folium import GeoJson, LayerControl
from folium.plugins import MeasureControl
# from shapely.geometry import MultiPolygon  # Only needed if you are converting polygons to multipolygons

# Step 1: Load Data
rail_data = pd.read_csv('annual_accident_probabilities.csv')
rail_data['geometry'] = rail_data['geometry'].apply(wkt.loads)
rails_gdf = gpd.GeoDataFrame(rail_data, geometry='geometry', crs="EPSG:4326")

tract_data = pd.read_csv('Merged_SVI_AAR_Data.csv')
tract_shapefile_gdf = gpd.read_file('Indiana.shp').to_crs("EPSG:4326")

tract_data['GEOID'] = tract_data['GEOID'].astype(str).str.zfill(11)
tract_shapefile_gdf['GEOID'] = tract_shapefile_gdf['GEOID'].astype(str).str.zfill(11)
tract_gdf = tract_shapefile_gdf.merge(
    tract_data[['GEOID', 'total_combined_risk', 'RPL_THEMES']],
    on='GEOID',
    how='left'
).fillna({'total_combined_risk': 0})

# Step 2: Scale Accident Probabilities
scaler = MinMaxScaler(feature_range=(1, 10))
rails_gdf['scaled_accident_probability'] = scaler.fit_transform(
    rails_gdf[['annual_accident_probability_percent']]
)

# Step 3: Combine with Census Tract Risk
joined_gdf = gpd.sjoin(rails_gdf, tract_gdf, how="left", predicate="intersects")

def aggregate_risks(group):
    return {
        'average_total_combined_risk': group['total_combined_risk'].mean(),
        'average_SVI_risk': group['RPL_THEMES'].mean(),
        'tract_ids': ','.join(group['GEOID'].astype(str).unique())
    }

aggregated = (
    joined_gdf.groupby('segment_id', group_keys=False)
    .apply(aggregate_risks)
    .apply(pd.Series)
    .reset_index()
)

rails_gdf = rails_gdf.merge(aggregated, on='segment_id', how='left')
rails_gdf['combined_vulnerability_score'] = (
    .5*rails_gdf['scaled_accident_probability'] + .25*rails_gdf['average_total_combined_risk'].fillna(0) + .25*rails_gdf['average_SVI_risk'].fillna(0)
)

# Step 4: Diffusion Zone Creation
rails_gdf = rails_gdf.to_crs("EPSG:26916")
rails_gdf['buffer'] = rails_gdf.geometry.buffer(5280)
rails_gdf = rails_gdf.to_crs("EPSG:4326")

buffer_gdf = gpd.GeoDataFrame(
    rails_gdf[['segment_id', 'combined_vulnerability_score', 'buffer']]
    .rename(columns={'buffer': 'geometry'}),
    geometry='geometry',
    crs="EPSG:4326"
)

# Convert Polygon to MultiPolygon if needed (often not required)
# tract_gdf['geometry'] = tract_gdf['geometry'].apply(lambda geom: MultiPolygon([geom]) if geom.geom_type == 'Polygon' else geom)
# buffer_gdf['geometry'] = buffer_gdf['geometry'].apply(lambda geom: MultiPolygon([geom]) if geom.geom_type == 'Polygon' else geom)

# Step 5: Visualization with Folium
m = folium.Map(location=[40.2672, -86.1349], zoom_start=7)

# Remove the 'buffer' column from rails_gdf if still present
if 'buffer' in rails_gdf.columns:
    rails_gdf = rails_gdf.drop(columns='buffer')

tract_geojson = json.loads(tract_gdf.to_json())
GeoJson(
    data=tract_geojson,
    style_function=lambda x: {
        'fillColor': 'red' if x['properties']['total_combined_risk'] > 8 else
                     'orange' if x['properties']['total_combined_risk'] > 5 else 'green',
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['GEOID', 'total_combined_risk'],
        aliases=['Census Tract:', 'Total Combined Risk:'],
        localize=True
    ),
    name="Census Tracts"
).add_to(m)

rails_geojson = json.loads(rails_gdf.to_json())
GeoJson(
    data=rails_geojson,
    style_function=lambda x: {
        'color': 'red' if x['properties']['combined_vulnerability_score'] > 8 else
                 'yellow' if x['properties']['combined_vulnerability_score'] > 5 else 'green',
        'weight': 2,
        'opacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['segment_id', 'combined_vulnerability_score'],
        aliases=['Segment ID:', 'Combined Vulnerability Score:'],
        localize=True
    ),
    name="Rail Segments"
).add_to(m)

buffer_geojson = json.loads(buffer_gdf.to_json())
GeoJson(
    data=buffer_geojson,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 0.5,
        'fillOpacity': 0.4
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['segment_id', 'combined_vulnerability_score'],
        aliases=['Segment ID:', 'Combined Vulnerability Score:'],
        localize=True
    ),
    name="Diffusion Zones"
).add_to(m)

folium.LayerControl().add_to(m)
m.add_child(MeasureControl())
m.save("combined_vulnerability_map.html")

# Step 7: Export Data
rails_gdf[['segment_id', 'scaled_accident_probability', 'average_total_combined_risk', 'combined_vulnerability_score']]\
    .to_csv("combined_vulnerability_scores.csv", index=False)
buffer_gdf.to_file("rail_diffusion_zones.geojson", driver='GeoJSON')
# No need to drop 'buffer' here since it's in buffer_gdf now, not rails_gdf
rails_gdf.to_file("rail_segments.geojson", driver='GeoJSON')
tract_gdf.to_file("tract_geometries.geojson", driver='GeoJSON')



In [6]:
import json
import pandas as pd
import geopandas as gpd
from shapely import wkt
from sklearn.preprocessing import MinMaxScaler
import folium
from folium import GeoJson, LayerControl
from folium.plugins import MeasureControl
# from shapely.geometry import MultiPolygon  # Only needed if you are converting polygons to multipolygons

# Step 1: Load Data
rail_data = pd.read_csv('annual_accident_probabilities.csv')
rail_data['geometry'] = rail_data['geometry'].apply(wkt.loads)
rails_gdf = gpd.GeoDataFrame(rail_data, geometry='geometry', crs="EPSG:4326")

tract_data = pd.read_csv('Merged_SVI_AAR_Data.csv')
tract_shapefile_gdf = gpd.read_file('Indiana.shp').to_crs("EPSG:4326")

tract_data['GEOID'] = tract_data['GEOID'].astype(str).str.zfill(11)
tract_shapefile_gdf['GEOID'] = tract_shapefile_gdf['GEOID'].astype(str).str.zfill(11)
tract_gdf = tract_shapefile_gdf.merge(
    tract_data[['GEOID', 'total_combined_risk', 'RPL_THEMES']],
    on='GEOID',
    how='left'
).fillna({'total_combined_risk': 0})

# Step 2: Scale Accident Probabilities
scaler = MinMaxScaler(feature_range=(1, 10))
rails_gdf['scaled_accident_probability'] = scaler.fit_transform(
    rails_gdf[['annual_accident_probability_percent']]
)

# Step 3: Combine with Census Tract Risk
joined_gdf = gpd.sjoin(rails_gdf, tract_gdf, how="left", predicate="intersects")

def aggregate_risks(group):
    return {
        'average_total_combined_risk': group['total_combined_risk'].mean(),
        'average_SVI_risk': group['RPL_THEMES'].mean(),
        'tract_ids': ','.join(group['GEOID'].astype(str).unique())
    }

aggregated = (
    joined_gdf.groupby('segment_id', group_keys=False)
    .apply(aggregate_risks)
    .apply(pd.Series)
    .reset_index()
)

rails_gdf = rails_gdf.merge(aggregated, on='segment_id', how='left')
rails_gdf['combined_vulnerability_score'] = (
    .5*rails_gdf['scaled_accident_probability'] + .25*rails_gdf['average_total_combined_risk'].fillna(0) + .25*rails_gdf['average_SVI_risk'].fillna(0)
)

# Step 4: Diffusion Zone Creation
# Reproject to a projected CRS for accurate buffering
rails_gdf = rails_gdf.to_crs("EPSG:26916")

# Use ~1 mile in meters (1609 m) instead of 5280 feet (as EPSG:26916 is in meters)
rails_gdf['buffer'] = rails_gdf.geometry.buffer(1609)

# Reproject back to geographic CRS (WGS84)
rails_gdf = rails_gdf.to_crs("EPSG:4326")

# Create buffer GeoDataFrame
buffer_gdf = gpd.GeoDataFrame(
    rails_gdf[['segment_id', 'combined_vulnerability_score', 'buffer']]
    .rename(columns={'buffer': 'geometry'}),
    geometry='geometry',
    crs="EPSG:4326"
)

# Step 4b: Rename columns to requested names after all calculations
# Rename tract-level score
tract_gdf = tract_gdf.rename(columns={'total_combined_risk': 'Commodity_Risk_Score'})

# Rename segment-level score
rails_gdf = rails_gdf.rename(
    columns={
        'combined_vulnerability_score': 'Segment_Risk_Score',
        'average_total_combined_risk': 'average_tract_vulnerability_score'
    }
)
buffer_gdf = buffer_gdf.rename(columns={'combined_vulnerability_score': 'Segment_Risk_Score'})

# Step 5: Visualization with Folium
m = folium.Map(location=[40.2672, -86.1349], zoom_start=7)

# Remove the 'buffer' column from rails_gdf if still present
if 'buffer' in rails_gdf.columns:
    rails_gdf = rails_gdf.drop(columns='buffer')

# Convert GeoDataFrames to GeoJSON-like dictionaries
tract_geojson = json.loads(tract_gdf.to_json())
rails_geojson = json.loads(rails_gdf.to_json())
buffer_geojson = json.loads(buffer_gdf.to_json())

# Add Census Tracts Layer
GeoJson(
    data=tract_geojson,
    style_function=lambda x: {
        'fillColor': 'red' if x['properties']['Commodity_Risk_Score'] > 8 else
                     'orange' if x['properties']['Commodity_Risk_Score'] > 5 else 'green',
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['GEOID', 'Commodity_Risk_Score'],
        aliases=['Census Tract:', 'Commodity Risk Score:'],
        localize=True
    ),
    name="Census Tracts"
).add_to(m)

# Add Rail Segments Layer
GeoJson(
    data=rails_geojson,
    style_function=lambda x: {
        'color': 'red' if x['properties']['Segment_Risk_Score'] > 8 else
                 'yellow' if x['properties']['Segment_Risk_Score'] > 5 else 'green',
        'weight': 2,
        'opacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['segment_id', 'Segment_Risk_Score'],
        aliases=['Segment ID:', 'Segment Risk Score:'],
        localize=True
    ),
    name="Rail Segments"
).add_to(m)

# Add Diffusion Zones Layer
GeoJson(
    data=buffer_geojson,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['segment_id', 'Segment_Risk_Score'],
        aliases=['Segment ID:', 'Segment Risk Score:'],
        localize=True
    ),
    name="Diffusion Zones"
).add_to(m)

folium.LayerControl().add_to(m)
m.add_child(MeasureControl())
m.save("combined_vulnerability_map.html")

# Step 7: Export Data
# Update the CSV and GeoJSON exports with renamed columns
rails_gdf[['segment_id', 'scaled_accident_probability', 'average_tract_vulnerability_score', 'Segment_Risk_Score']]\
    .to_csv("combined_vulnerability_scores.csv", index=False)

buffer_gdf.to_file("rail_diffusion_zones.geojson", driver='GeoJSON')
rails_gdf.to_file("rail_segments.geojson", driver='GeoJSON')
tract_gdf.to_file("tract_geometries.geojson", driver='GeoJSON') 
